In [ ]:
import numpy as np
import os.path
import pandas
import matplotlib
import scipy.signal
import scipy.constants

%matplotlib inline
import matplotlib.pyplot as plt
from matplotlib import gridspec

# import mpld3
# from mpld3 import plugins


# New version of this notebook
# First todo: clean up code. This mess is likely to introduce a mistake.



## Functions to load xviewer csv (from binary scope data) and scope csv

In [ ]:
def read_scope_xviewer_csv(filename):
    """ Read csv files exported using Yokogawa XViewer software
        
        Update 11-09-2017: auto-detect number of columns.        
    """
    headerRows = 10        
    with open(filename) as f:
        for x in range(headerRows):
            line = f.readline()            
            if "HResolution" in line:
                i1 = line.find(',')+1
                i2 = line.find(',',i1)                
                timeResolution = float( line[i1:i2].strip(' \t\n\r') )        
                numberOfCols = line.count(',')    
    data = pandas.read_csv(filename,header=None,skiprows=headerRows,usecols=range(1,numberOfCols)).values
    return data, timeResolution


def read_scope_csv(filename):
    """ Read csv files as stored by Yokogawa Ossiloscope.                     
    """    
    headerRows = 16        
    with open(filename) as f:
        for x in range(headerRows):
            line = f.readline()            
            if "HResolution" in line:                
                timeResolution = float( line[line.rfind(',')+1:-1].strip(' \t\n\r') )    
    data = pandas.read_csv(filename,header=None,skiprows=headerRows).values[:,1:-1]
    return data, timeResolution



## Functions to analize vibration time-traces

In [ ]:
from scipy.optimize import minimize
from scipy import interpolate

def lorentzian(x,p):    
    """
    Lorenzian function p[0] is center, p[1] is fwhm, p[2] is maximum intensity
    Similar to http://mathworld.wolfram.com/LorentzianFunction.html, but with p[2] maximum intensity, NOT integrated intensity of one.
    """
    return p[2] * (0.5 * p[1])**2 / ( ( x - p[0] )**2 + ( 0.5 * p[1] )**2 )

def lorentz_find_x(lorentzFwhm, lorentzMax, lorentzValue):    
    """ For lorenzian function y=f(x) with lorentzFwhm and lorentzMax centered at x=0, find x for y=lorentzValue. """
    def fun(x):
        return ( lorentzian( x, [0, lorentzFwhm, lorentzMax] ) -  lorentzValue ) ** 2    
    res = minimize( fun, lorentzFwhm,  method='Nelder-Mead', bounds=( (0, None ), ) )
    if not( res.success ):
        print 'lorentzFindX convergence error: ', res.message       
    return res.x[0]
    
def lorentz_find_flank_steepness(lorentzFwhm, lorentzMax, x):    
    """ Return steepness of flank for of lorentz dy/dx for given x. Assume lorentz centered at x=0. """  
    return lorentzMax * 8.0 * lorentzFwhm**2 * x / ( 4.0 * x**2 + lorentzFwhm**2 )**2
    

def linear_voltage_to_detuning(d, t, lorentzFwhm, lorentzMaxV):
    flankSteepness = lorentz_find_flank_steepness( lorentzFwhm, lorentzMaxV, lorentz_find_x( lorentzFwhm, lorentzMaxV, meanVInD ) )
    dHz = d / flankSteepness    
    return dHz

def rms_deviation( x ):
    """ return root-mean-square value of signal, with mean substracted """
    return np.sqrt(np.mean( (x - np.mean(x))**2))
    
def lorentz_voltage_to_detuning(d, t, lorentzFwhm, lorentzMaxV, doPlot=False ):    
    meanVInD = np.mean( d ) 
    minVInD = np.amin(d)    
    # Compute vibrations in Hz by first applying lorentzian function as lookup table V -> dHz
    minDFreqInD = np.absolute( lorentz_find_x( lorentzFwhm, lorentzMaxV, minVInD ) )
    lorentzDFreq = np.linspace( -minDFreqInD-1e9, 0, 10000 )        
    lorentzV = lorentzian( lorentzDFreq, [0, lorentzFwhm, lorentzMaxV] )    
    # print minDFreqInD, np.amin( lorentzV ),  np.amax( lorentzV )
    dHz = interpolate.interp1d(lorentzV, lorentzDFreq)(d)  # will cause error if signal > lorentzMaxV  or if something went wrong with minDFreqInD
    # Print and plot results
    if doPlot:   
        i1 = 0
        i2 = min( 50000, len(d) )
        fig = plt.figure(figsize=(14, 6))
        plt.subplot(131)
        plt.plot( t[i1:i2], d[i1:i2] )
        plt.xlabel( 'time (s)' )
        plt.ylabel( 'transmission (V)' )
        plt.grid()
        plt.subplot(132)
        plt.plot( t[i1:i2], dHz[i1:i2]*1e-9 )
        plt.xlabel( 'time (s)' )
        plt.ylabel( 'cavity detuning (GHz)' )
        plt.grid()
        plt.subplot(133)
        plt.plot( lorentzDFreq*1e-9, lorentzV )
        plt.xlabel( 'cavity detuning GHz' )
        plt.ylabel( 'V' )
        plt.grid()
        plt.show()
    return dHz
            
def calc_cum_pow( freq_pow, power, min_freq=0.1, max_freq=20e3 ):
    """ Compute spectra and cumtrapz 
    
        Input:
          freq_pow - frequency axis, 1D array
          power    - power, 1D array (as function of frequency)        
          min_freq - minimum frequency in analysis. DC power is power in frequencys below min_freq. Hz.
          max_freq - maximum frequency 
        
        Output:
          cum_freqs - frequency axis for cum_pow
          cum_pow   - cummulative/integrated power spectrum, using trapz
          full_pow  - power integrated over all frequencies
          dc_pow    - power in frequencies below min_freq
        
        
    
    
    """
    
    # min_freq = 0.1  # we do not take frequencies below 0.1 Hz into account.
    # max_freq = 20e3

    min_ind = np.argmin(np.abs(freq_pow-min_freq))
    max_ind = np.argmin(np.abs(freq_pow-max_freq))

    full_pow = scipy.integrate.trapz(power,freq_pow,axis=0)
    dc_pow = scipy.integrate.trapz(power[0:min_ind],freq_pow[0:min_ind],axis=0)

    cum_freqs = freq_pow[min_ind+1:max_ind]      
    cum_pow = scipy.integrate.cumtrapz( power[min_ind:max_ind], freq_pow[min_ind:max_ind] )
    cum_pow = cum_pow / (full_pow-dc_pow)  
    
    return cum_freqs, cum_pow, full_pow, dc_pow    

## Measurements 14 September 2017 - Cavity length 20 um

In [ ]:
# Load files in memory.

dataDir = '/Users/wjwesterveld/Documents/Temp_CAV1_data/20170914/scope'
fileNames = [ 'SCOPE_013.wdf.csv', 'SCOPE_015.wdf.csv', 'SCOPE_017.wdf.csv', 'SCOPE_019.wdf.csv' ]
dataDict = {}

for fileName in fileNames:
    if fileName in dataDict:
        continue
    data, timeResolution = read_scope_xviewer_csv( os.path.join(dataDir,fileName) )
    time = timeResolution * np.arange(data.shape[0]) 
    print 'time resolution {} us (sampling frequency {} kHz), trace length {} s, data length {}'.format( \
        timeResolution * 1e6, 1.0/timeResolution* 1e-3, timeResolution*len(data), data.shape[0] )   
    dataDict[fileName] = data


In [ ]:
# Plot first check of files

for fileName in fileNames:    
    data = dataDict[fileName]
    i1 = 0
    i2 = np.argmax( time > 1 )
    plt.figure(figsize=(16, 8))
    plt.plot( time[i1:i2], data[i1:i2,0] )
    plt.grid()
    plt.xlabel( 'time (s)' )
    plt.ylabel( 'V' )
    plt.title( fileName )
    plt.show()  

for fileName in fileNames:    
    data = dataDict[fileName]    
    plt.hist(data[:,0],20,label=fileName)
plt.legend()
plt.grid()
plt.show()

for fileName in fileNames:    
    data = dataDict[fileName]    
    print 'filename {}, min {} V, max {} V'.format( fileName, np.amin( data[:,0] ), np.amax( data[:,0] ) )    


In [ ]:
fileName = 'SCOPE_019.wdf.csv'

data = dataDict[ fileName ]

# Analysis
lorentzFwhm = 23e9      # theortical estimate from specified mirror reflectivities and clipping losses
lorentzMaxV = 0.1505312 # maximum in measurements
cavLength = 19.7e-6
cavWavelength = 637e-9
cavFreq = scipy.constants.c / cavWavelength

# Histogram
plt.hist(data[:,0],20,label=fileName)
plt.legend()
plt.grid()
plt.show()

# Vibrations
df = lorentz_voltage_to_detuning( data[:,0], time, lorentzFwhm, lorentzMaxV, doPlot=True )
dfRMS = rms_deviation( scipy.signal.detrend( df ) )
dlRMS = dfRMS * cavLength / cavFreq
print 'rms vibrations {:.3f} V, {:.3f} GHz, {:.3f} nm using Lorenzian look-up'.format( rms_deviation( data[:,0] ), dfRMS*1e-9 , dlRMS*1e9 )                





In [ ]:
# compute spectrum
dl = df * cavLength / cavFreq
freq_pow, power = scipy.signal.welch( dl, axis = 0, fs = 1/timeResolution, nperseg = len(dl), detrend = 'linear' )


In [ ]:
# integrate spectrum

min_freq = 0.1  # we do not take frequencies below 0.1 Hz into account.
max_freq = 20e3

min_ind = np.argmin(np.abs(freq_pow-min_freq))
max_ind = np.argmin(np.abs(freq_pow-max_freq))

full_pow = scipy.integrate.trapz(power,freq_pow,axis=0)
dc_pow = scipy.integrate.trapz(power[0:min_ind],freq_pow[0:min_ind],axis=0)

cum_freqs = freq_pow[min_ind+1:max_ind]      
cum_pow = scipy.integrate.cumtrapz( power[min_ind:max_ind], freq_pow[min_ind:max_ind] )
cum_pow = cum_pow / (full_pow-dc_pow)  

In [ ]:
plt.figure(figsize=(12, 12))
plt.subplot(211)
plt.plot(freq_pow[min_ind:max_ind], 10.0*np.log10( power[min_ind:max_ind] ) )
plt.grid()
plt.xlabel('freq (Hz)')
plt.ylabel('Noise power (m**2 in dB)')
plt.subplot(212)
plt.plot(cum_freqs, cum_pow)
plt.title( 'full noise power {:.3f} nm/sqrt(Hz), dc frequency {} Hz, relative DC power {:.3f}'.format( np.sqrt( full_pow ) *1e9, min_freq, dc_pow / full_pow )  )
plt.grid()
plt.xlabel('freq (Hz)')
plt.ylabel('Integrated noise power, relative (m**2)')
plt.show()

  


In [ ]:
plt.figure(figsize=(12, 12))
plt.plot( cum_freqs, plot1, label='#017, 18 GHz lw' )
plt.plot( cum_freqs, plot2, label='#017, 23 GHz lw', linestyle=':' )
plt.plot( cum_freqs, plot3, label='#019, 18 GHz lw' )
plt.plot( cum_freqs, plot4, label='#019, 23 GHz lw', linestyle=':' )
plt.xlim( 0, 1000 )
plt.grid()
plt.xlabel('freq (Hz)')
plt.ylabel('Integrated noise power, relative (m**2)')
plt.legend()
plt.show()

## Measurements 18 September 2017 - Cavity length 20 um

In [ ]:
dataDir = '/Users/wjwesterveld/Documents/Temp_CAV1_data/20170918/scope'


fileNames = [ 'SCOPE_006.wdf.csv', 'SCOPE_007.wdf.csv', 'SCOPE_008.wdf.csv', 'SCOPE_009.wdf.csv', 'SCOPE_010.wdf.csv', 'SCOPE_011.wdf.csv', 'SCOPE_012.wdf.csv', 'SCOPE_020.wdf.csv' ]
# fileNames = [ 'SCOPE_023.wdf.csv', 'SCOPE_025.wdf.csv', 'SCOPE_026.wdf.csv' ]
# fileNames = [ 'SCOPE_006.wdf.csv', 'SCOPE_013.wdf.csv', 'SCOPE_014.wdf.csv', 'SCOPE_015.wdf.csv', 'SCOPE_016.wdf.csv', 'SCOPE_017.wdf.csv', 'SCOPE_018.wdf.csv', 'SCOPE_019.wdf.csv' ]

if not( 'dataDict' in locals() ):
    dataDict = {}
    timeResolutionDict = {}

for fileName in fileNames:
    if fileName in dataDict:
        continue
    data, timeResolution = read_scope_xviewer_csv( os.path.join(dataDir,fileName) )
    time = timeResolution * np.arange(data.shape[0]) 
    print 'time resolution {} us (sampling frequency {} kHz), trace length {} s, data length {}'.format( \
        timeResolution * 1e6, 1.0/timeResolution* 1e-3, timeResolution*len(data), data.shape[0] )   
    dataDict[fileName] = data
    timeResolutionDict[fileName] = timeResolution


In [ ]:
for fileName in fileNames:    
    data = dataDict[fileName]
    i1 = 0
    i2 = np.argmax( time > 1 )
    plt.figure(figsize=(16, 8))
    plt.plot( time[i1:i2], data[i1:i2,0] )
    plt.grid()
    plt.xlabel( 'time (s)' )
    plt.ylabel( 'V' )
    plt.title( fileName )
    plt.show()  

for fileName in fileNames:    
    data = dataDict[fileName]    
    plt.hist(data[:,0],20,label=fileName)
plt.legend()
plt.grid()
plt.show()

for fileName in fileNames:    
    data = dataDict[fileName]    
    print 'filename {}, min {:10.3f} V, max {:10.3f} V'.format( fileName, np.amin( data[:,0] ), np.amax( data[:,0] ) )  



In [ ]:
# Compute dl and RMS values

# First filename is not taken into accout hereafter because it is expected to be for min/max finding only.

dlDict = {}

for fileName in fileNames[1:]:    
    data = dataDict[fileName]    
    
    # Analysis    
    lorentzFwhm = 18.0e9        # theortical estimate from specified mirror reflectivities and clipping losses
    lorentzMaxV = 0.0863000001  # maximum in measurements, #006 to #022
    cavLength = 19.7e-6
    # lorentzMaxV = 0.177         # maximum in measurements, #023 to #025        
    # cavLength = 19.3e-6
    cavWavelength = 637e-9
    cavFreq = scipy.constants.c / cavWavelength

    # Vibrations
    df = lorentz_voltage_to_detuning( data[:,0], time, lorentzFwhm, lorentzMaxV, doPlot=True )
    dfRMS = rms_deviation( scipy.signal.detrend( df ) )
    dlRMS = dfRMS * cavLength / cavFreq        
    print '{} (lw {:.1f} GHz) rms vibrations {:.3f} V, {:.3f} GHz, {:.3f} nm'.format( fileName, lorentzFwhm*1e-9, rms_deviation( data[:,0] ), dfRMS*1e-9 , dlRMS*1e9 )                    
                
    dlDict[fileName] = df * cavLength / cavFreq
                
    
    
#013 0.144 nm
#014 0.149 nm
#015 0.169 nm
#016 0.166 nm

#017	0.122 nm
#018	0.127
#019	0.118

In [ ]:
# Compute spectra and cumtrapz

plt.figure(figsize=(12, 12))

powerDict = {}
freqPowDict = {}  

for fileName in fileNames[1:]:    
    dl = dlDict[fileName]  
    freqPow, power = scipy.signal.welch( dl, axis = 0, fs = 1/timeResolution, nperseg = len(dl), detrend = 'linear' )
    powerDict[fileName] = power
    freqPowDict[fileName] = freqPow
        
    cumFreqs, cumPow, fullPow, dcPow = calc_cum_pow( freqPow, power, max_freq=20e3  )
       
    plt.plot( cumFreqs*1e-3, cumPow, label=fileName )        
    print '{}, full noise power {:.3f} nm/sqrt(Hz), dc frequency {} Hz, relative DC power {:.3f}'.format( fileName, np.sqrt( fullPow ) *1e9, 0.1, dcPow / fullPow ) 
    
plt.grid()
plt.xlabel('freq (kHz)')
plt.ylabel('Integrated noise power, relative (m**2)')
plt.legend()
plt.show()
    

In [ ]:
# analyze drift

from scipy.signal import butter, filtfilt

def butter_lowpass(cutoff, fs, order=5):
    nyq = 0.5 * fs
    normal_cutoff = cutoff / nyq
    b, a = butter(order, normal_cutoff, btype='low', analog=False)
    return b, a

def butter_lowpass_filtfilt(data, cutoff, fs, order=5):
    b, a = butter_lowpass(cutoff, fs, order=order)    
    y = filtfilt(b, a, data)
    return y

plt.figure(figsize=(12, 9))

for fileName in fileNames[1:]:    
    dl = dlDict[fileName]  
    timeResolution = timeResolutionDict[fileName]
    time = timeResolution * np.arange(dl.shape[0])         
    dlSmooth = butter_lowpass_filtfilt(data=dl, cutoff=0.1, fs=0.1/timeResolution,order=3)            
    
    p = np.polyfit( time[::100], dlSmooth[::100], 1)    
    plt.plot( time[::100], dlSmooth[::100]*1e9 - np.mean( dlSmooth[::100]*1e9 ), label='{} {:.3f} nm/hr'.format( fileName, p[0]*1e9*3600 ) )    
    
    
plt.grid()
plt.title( 'low-pass 0.1 Hz, 3rd butter' )
plt.xlabel( 'time (s)' )
plt.ylabel( 'dl (nm)' )
plt.legend()
plt.show()



In [ ]:
# Influence of photo-diode noise
# Plot relative noise around 50 Hz as function of mean voltage in time-trace.
# Shows clear correlation.

dataMeans = np.zeros( len( fileNames) - 1)
dlRelative50Hz = np.zeros( len( fileNames) - 1)
dlRMSs = np.zeros( len( fileNames) - 1)

for i in range( len( fileNames ) - 1 ):  
    fileName = fileNames[i+1]
    data = dataDict[fileName]
    dataMeans[i] = np.mean( data[:,0] )
    dl = dlDict[fileName] 
    dlRMSs[i] = rms_deviation( scipy.signal.detrend( dl ) )
    power = powerDict[fileName]
    freqPow = freqPowDict[fileName]    
    i1 = np.argmin( np.abs( freqPow - 45.0 ) )
    i2 = np.argmin( np.abs( freqPow - 55.0 ) )
    fullPow = scipy.integrate.trapz(power,freqPow,axis=0)
    pow50Hz = scipy.integrate.trapz(power[i1:i2],freqPow[i1:i2],axis=0)           
    dlRelative50Hz[i] = pow50Hz / fullPow
    

plt.plot( dataMeans, dlRelative50Hz, '.' )
plt.xlabel( 'mean voltage V ')
plt.ylabel( 'noise around 50 Hz, relative' )
plt.grid()
plt.show()    
    
    
plt.plot( dataMeans, dlRMSs*1e9, '.' )
plt.xlabel( 'mean voltage (V)')
plt.ylabel( 'rms vibrations (nm)' )
plt.grid()
plt.show()        

In [ ]:
## More detailed analysis of spectrum
#
# Single time-trace only.
#
# Load data, time, dl, power, freqPow

fileName = 'SCOPE_020.wdf.csv'
data = dataDict[fileName]
timeResolution = timeResolutionDict[fileName]
dl = dlDict[fileName]
power = powerDict[fileName]
freqPow = freqPowDict[fileName] 
time = timeResolution * np.arange(data.shape[0]) 

In [ ]:
## More detailed analysis of spectrum


# Plot histogram
plt.hist( dl*1e9, 100, label='dl')
plt.hist( scipy.signal.detrend( dl )*1e9, 100, label='dl, linear detrend')
plt.legend()
plt.xlabel( 'detuning (nm)' )
plt.grid()
plt.show()

# Detrend dl
dlDetrended = scipy.signal.detrend( dl )
hist, binEdges = np.histogram( dlDetrended, 250)
binCenters = 0.5 * ( binEdges[1:] + binEdges[:-1] )

# Fit gauss and lorentzian curves to histogram
from scipy.optimize import curve_fit
from scipy import asarray as ar,exp

def gaus(x,a,x0,sigma):
    return a*exp(-(x-x0)**2/(2*sigma**2))

def fit_gauss(x, y, sigmaGuess):
    popt,pcov = curve_fit(gaus,x,y,p0=[ np.max(y), x[ np.argmax(y) ], sigmaGuess ] )        
    return popt

def lorentzian2(x, p1, p2, p3):
    return lorentzian(x, [p1, p2, p3])

def fit_lorentz(x, y, pGuess):
    popt,pcov = curve_fit(lorentzian2,x,y,p0=pGuess )        
    return popt

sigmaGuess = rms_deviation( dlDetrended )
popt = fit_gauss( binCenters, hist, sigmaGuess )
x = np.linspace( binCenters[0], binCenters[-1], 1000 )
yFit = gaus( x, *popt )

popt2 = fit_lorentz( binCenters, hist, [popt[1], popt[2], popt[0]] )
yFit2 = lorentzian2( x, *popt2 )

plt.plot( binCenters*1e9, hist )
plt.plot( x*1e9, yFit, ':', label='gauss, sigma {:3f} nm'.format( popt[2]*1e9 ) )
plt.plot( x*1e9, yFit2, ':', label='lorentz, fwhm {:3f} nm'.format( popt2[1]*1e9 ) )
plt.xlabel( 'detuning (nm)' )
plt.title( '{}, linear detrend, rms vibrations {:.3f} nm'.format( fileName, sigmaGuess*1e9 ) )
plt.legend()
plt.grid()
plt.show()







In [ ]:
# Smooth signal
from scipy.signal import butter, filtfilt

def butter_lowpass(cutoff, fs, order=5):
    nyq = 0.5 * fs
    normal_cutoff = cutoff / nyq
    b, a = butter(order, normal_cutoff, btype='low', analog=False)
    return b, a

def butter_lowpass_filtfilt(data, cutoff, fs, order=5):
    b, a = butter_lowpass(cutoff, fs, order=order)    
    y = filtfilt(b, a, data)
    return y

dlSmooth1 = butter_lowpass_filtfilt(data=dl, cutoff=1.0, fs=0.1/timeResolution,order=3)
dlSmooth2 = butter_lowpass_filtfilt(data=dl, cutoff=0.1, fs=0.1/timeResolution,order=3)


# Plot time traces
# 1s, at various starting times
for timeStart in [0, 10, 20, 30]:
    i1 = np.argmax( time > timeStart )
    i2 = np.argmax( time > timeStart + 1.0 )
    plt.figure(figsize=(12, 8))
    plt.plot( time[i1:i2], dl[i1:i2]*1e9 )    
    plt.plot( time[i1:i2], dlSmooth1[i1:i2]*1e9, ':', label='low-pass 1.0 Hz, 3rd order butter' )    
    plt.plot( time[i1:i2], dlSmooth2[i1:i2]*1e9, ':', label='low-pass 0.1 Hz, 3rd order butter' )    
    plt.xlabel( 'time (s)' )
    plt.ylabel( 'dl (nm)' )
    plt.title( fileName )
    plt.legend()
    plt.grid()
    plt.show()



In [ ]:

timeStart = 0.105

for w in [10, 1, 0.1, 0.01, 0.001 ]:
    i1 = np.argmax( time > timeStart )
    i2 = np.argmax( time > (timeStart + w) )        
    plt.plot( time[i1:i2] - time[i1], dl[i1:i2]*1e9)    
    plt.xlabel( 'time (s)' )
    plt.ylabel( 'dl (nm)' )
    plt.grid()
    plt.show()
    
timeStart = 0.305
for w in [10, 1, 0.1, 0.01, 0.001 ]:
    i1 = np.argmax( time > timeStart )
    i2 = np.argmax( time > (timeStart + w) )        
    plt.plot( time[i1:i2] - time[i1] + 0.20, dl[i1:i2]*1e9)    
    plt.xlabel( 'time (s)' )
    plt.ylabel( 'dl (nm)' )
    plt.grid()
    plt.show()
    
    

In [ ]:

# Plot time traces
# 1s, at various starting times
for timeStart in [0, 10, 20, 30]:
    i1 = np.argmax( time > timeStart )
    i2 = np.argmax( time > timeStart + 1.0 )
    plt.figure(figsize=(12, 8))
    plt.plot( time[i1:i2], dl[i1:i2]*1e9 )    
    plt.plot( time[i1:i2], dlSmooth1[i1:i2]*1e9, ':', label='low-pass 1.0 Hz, 3rd order butter' )    
    plt.plot( time[i1:i2], dlSmooth2[i1:i2]*1e9, ':', label='low-pass 0.1 Hz, 3rd order butter' )    
    plt.xlabel( 'time (s)' )
    plt.ylabel( 'dl (nm)' )
    plt.title( fileName )
    plt.legend()
    plt.grid()
    plt.show()

In [ ]:
# Plot spectrum (again, now only one file to get grid of )


cumFreqs, cumPow, fullPow, dcPow = calc_cum_pow( freqPow, power, max_freq=20e3  )

min_ind = np.argmax( freqPow > 0.1 )
max_ind = np.argmax( freqPow > 20e3 )

plt.figure(figsize=(12, 12))
plt.subplot(211)
plt.plot( freqPow[min_ind:max_ind], 10.0*np.log10( power[min_ind:max_ind] ) )
plt.xlabel('freq (Hz)')
plt.ylabel('Noise power (m**2 in dB)')
plt.grid()
plt.subplot(212)
plt.plot( cumFreqs*1e-3, cumPow )        
plt.xlabel('freq (kHz)')
plt.ylabel('Integrated noise power, relative (m**2)')
plt.grid()
plt.show()

print '{}, full noise power {:.3f} nm/sqrt(Hz), dc frequency {} Hz, relative DC power {:.3f}'.format( fileName, np.sqrt( fullPow ) *1e9, 0.1, dcPow / fullPow ) 

In [ ]:
maxFreq = 7.5e3

cumFreqs, cumPow, fullPow, dcPow = calc_cum_pow( freqPow, power, max_freq=maxFreq  )

min_ind = np.argmax( freqPow > 0.1 )
max_ind = np.argmax( freqPow > maxFreq )

plt.plot( cumFreqs*1e-3, cumPow )  
plt.xlim( [5, 7.5] )
plt.ylim( [0.4, 0.8] )
plt.xlabel('freq (kHz)')
plt.ylabel('Integrated noise power, relative (m**2)')
plt.grid()
plt.show()

In [ ]:
# Influence of photo-diode noise

dataDir = '/Users/wjwesterveld/Documents/Temp_CAV1_data/20170918/scope'
if ( 'fileName' in locals() ) or ( fileName != 'SCOPE_022.wdf.csv' ):
    fileName = 'SCOPE_022.wdf.csv'
    data, timeResolution = read_scope_xviewer_csv( os.path.join(dataDir,fileName) )
    time = timeResolution * np.arange(data.shape[0]) 
    print 'time resolution {} us (sampling frequency {} kHz), trace length {} s, data length {}'.format( \
        timeResolution * 1e6, 1.0/timeResolution* 1e-3, timeResolution*len(data), data.shape[0] )   

i1 = 0
i2 = np.argmax( time > 1 )
plt.figure(figsize=(16, 8))
plt.subplot(121)
plt.plot( time[i1:i2], data[i1:i2,0] )
plt.grid()
plt.xlabel( 'time (s)' )
plt.ylabel( 'V' )
plt.title( fileName )
plt.subplot(122) 
plt.hist(data[:,0],20,label=fileName)
plt.legend()
plt.grid()
plt.show()
print 'filename {}, min {:10.6f} V, max {:10.6f} V'.format( fileName, np.amin( data[:,0] ), np.amax( data[:,0] ) )  

dataZeroMean = data[:,0] - np.mean( data[:,0] )

plt.figure(figsize=(16, 8))
plt.subplot(121)
plt.plot( time[i1:i2], dataZeroMean[i1:i2] )
plt.grid()
plt.xlabel( 'time (s)' )
plt.ylabel( 'V' )
plt.title( fileName )
plt.subplot(122) 
plt.hist(dataZeroMean,20,label=fileName)
plt.legend()
plt.grid()
plt.show()
print 'filename {} zero-mean, min {:10.6f} V, max {:10.6f} V'.format( fileName, np.amin( dataZeroMean ), np.amax( dataZeroMean ) )  



    
# Cavity settings     
lorentzFwhm = 18.0e9        # theortical estimate from specified mirror reflectivities and clipping losses
lorentzMaxV = 0.0863000001  # maximum in measurements 
cavLength = 19.7e-6
cavWavelength = 637e-9
cavFreq = scipy.constants.c / cavWavelength

# Noise analysis
noiseOffsets = np.linspace( 0.004, 0.080, 15 )
noisedlRMSs = np.zeros( len( noiseOffsets ))


for i in range( len( noiseOffsets ) ):
    # Compute influence of noise at given voltage.
    # Noise is photdiode noise only, with voltage offeset to place it at various positions of lorentz flank.
    d =  dataZeroMean + noiseOffsets[i] 
    df = lorentz_voltage_to_detuning( d, time, lorentzFwhm, lorentzMaxV, doPlot=False )
    dfRMS = rms_deviation( scipy.signal.detrend( df ) )
    dlRMS = dfRMS * cavLength / cavFreq        
    print 'noise offset {:.3f} V, rms vibrations {:.3f} nm'.format( noiseOffsets[i], dlRMS*1e9 )                    
    noisedlRMSs[i] = dlRMS
    
    # spectrum
    dl = df * cavLength / cavFreq
    freqPow, power = scipy.signal.welch( dl, axis = 0, fs = 1/timeResolution, nperseg = len(dl), detrend = 'linear' )      
    cumFreqs, cumPow, fullPow, dcPow = calc_cum_pow( freqPow, power, max_freq=500  )
    plt.plot( cumFreqs*1e-3, cumPow, label='offset {:.3f} V'.format( noiseOffsets[i] ) )  
    
plt.grid()
plt.xlabel('freq (kHz)')
plt.ylabel('Integrated noise power, relative (m**2)')
plt.legend()
plt.show()    
    
plt.plot( noiseOffsets, noisedlRMSs*1e9 )
plt.title( 'noise RMS {:.3f} V '.format( rms_deviation( scipy.signal.detrend( d ) ) ) )
plt.xlabel( 'noise mean (V)' )
plt.ylabel( 'noise in length RMS (nm)' )
plt.grid()
plt.show()

    

    

In [ ]:
print cavFreq

print 100e9 * 10e-6 / cavFreq * 1e9 


In [ ]:
# Todo: influence of noise

# Todo: influence of drift? 

# Todo: 17, 18, 19

# Todo: 23, 25, 26, 27 (black)